<a href="https://colab.research.google.com/github/jeffersonramelo/Paper-pos-doc/blob/main/relato_integrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/relato integrado completo2.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)

excluir setor financeiro e empresas com PL negativo, numeros faltante e transformar eventuais caracteres em números

In [3]:
# Agora, vamos remover as linhas onde 'd9' seja igual a 1
dados = dados[dados['d11'] != 1]
# Converte a coluna 'PL' para números (float) e remove as linhas onde 'PL' seja negativo
dados['PL'] = pd.to_numeric(dados['PL'], errors='coerce')  # 'coerce' trata strings não numéricas como NaN
dados = dados[dados['PL'] > 0]

# Agora, vamos remover as linhas onde 'VM30Abril11' seja igual a '-'
dados = dados[dados['VM30Abrilt+1'] != '-']
dados = dados[dados['PL'] != '-']
dados = dados[dados['LL'] != '-']
dados = dados[dados['AT'] != '-']
dados = dados[dados['AT'] != '0']
dados = dados[dados['at-1'] != '-']
dados = dados[dados['at-1'] != '0']

#transforma as variáveis em números
dados['at-1'] = pd.to_numeric(dados['at-1'], errors='coerce')
dados['AT'] = pd.to_numeric(dados['AT'], errors='coerce')
dados['LL'] = pd.to_numeric(dados['LL'], errors='coerce')
dados['VM30Abrilt+1'] = pd.to_numeric(dados['VM30Abrilt+1'], errors='coerce')
dados['PL'] = pd.to_numeric(dados['PL'], errors='coerce')

# Lista de variáveis que você deseja escalonar
variaveis = ['VM30Abrilt+1', 'PL', 'LL', 'at-1', 'AT']

# Verifique o tipo de dados de cada coluna
for coluna in variaveis:
    print(f"Tipo de dados da coluna {coluna}: {dados[coluna].dtype}")


Tipo de dados da coluna VM30Abrilt+1: float64
Tipo de dados da coluna PL: float64
Tipo de dados da coluna LL: float64
Tipo de dados da coluna at-1: float64
Tipo de dados da coluna AT: float64


<ipython-input-3-513d05fd18a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['PL'] = pd.to_numeric(dados['PL'], errors='coerce')  # 'coerce' trata strings não numéricas como NaN


ESCALONAMENTO DAS VARIÁVEIS

In [1]:
variaveis = ['VM30Abrilt+1', 'PL', 'LL']

for coluna in variaveis:
    dados[coluna] = dados[coluna] / dados['at-1']


NameError: ignored

WINSORIZAÇÃO: tratar outliers: A winsorização ao nível de 1% envolve substituir os 1% dos valores mais extremos por valores no limite do intervalo aceitável, ou seja, os 1% menores e os 1% maiores dos valores.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import mstats

# Supondo que seu DataFrame se chame "dados"

# Lista de variáveis para winsorizar
variaveis_winsorizar = ['VM30Abrilt+1', 'PL', 'LL', 'at-1', 'AT']

# Converter as colunas para números, tratando não numéricos como NaN
for coluna in variaveis_winsorizar:
    dados[coluna] = pd.to_numeric(dados[coluna], errors='coerce')

# Aplicar winsorização de 1% nas variáveis
for coluna in variaveis_winsorizar:
    dados[coluna] = mstats.winsorize(dados[coluna], limits=[0.01, 0.01])

# Agora, as variáveis especificadas foram winsorizadas


In [5]:
import pandas as pd
import statsmodels.api as sm

# Suponha que você já tenha seu DataFrame 'dados' com as variáveis PL, LL, VM30Abrilt+1, t e ri.

# Vamos criar variáveis de interação entre 't' e 'ri' para calcular o efeito DID.
dados['t_ri'] = dados['t'] * dados['ri']

model = sm.OLS(dados['VM30Abrilt+1'], sm.add_constant(dados[['PL', 'LL', 'ri', 't', 't_ri']])).fit()

# Visualize os resultados do modelo.
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:           VM30Abrilt+1   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     703.7
Date:                Tue, 24 Oct 2023   Prob (F-statistic):               0.00
Time:                        13:44:39   Log-Likelihood:                -23115.
No. Observations:                1266   AIC:                         4.624e+04
Df Residuals:                    1260   BIC:                         4.627e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.363e+09   7.02e+08      1.940      0.0